In [25]:
import gtsam
import cv2
import numpy as np
import yaml

In [26]:
with open('out.yaml') as f:
    data = yaml.load(f)
    
object_points = np.float32(data['object_points'])
wTh_list = [gtsam.Pose3(
                gtsam.Rot3.Rodrigues(v['wTh']['rvec']), 
                v['wTh']['tvec']) 
            for v in data['views']]
image_points = np.float32([v['image_points'] for v in data['views']])

/home/tuwuhs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


# Reference implementation

In [27]:
def as_nparray(node):
    return np.array(node['data']).reshape([node['rows'], node['cols']])

with open('camera_info.yaml') as f:
    data = yaml.load(f)
K = as_nparray(data['camera_matrix'])
dist = as_nparray(data['distortion_coefficients'])

/home/tuwuhs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  """


In [29]:
## Tsai (VISP)
hTe = gtsam.Pose3(gtsam.Rot3(
     0.017563,  0.0236099, 0.999567, 
    -0.700223, -0.713329,  0.0291522, 
     0.713708, -0.700432,  0.00400403), 
    [0.0867232, 0.135053,  0.0780596])
oTw = gtsam.Pose3(gtsam.Rot3(
     0.0931247, -0.995408,  0.0221433,
     0.0944387,  0.0309705, 0.995049, 
    -0.991166,  -0.0905724, 0.0968892),
    [0.0643109, -0.849175, 1.14672])

rrmse = 0
count = 0
for i in range(len(wTh_list)):
    oTe = oTw.compose(wTh_list[i]).compose(hTe)
    eTo = oTe.inverse()
    reprojected = cv2.projectPoints(
        object_points,
        gtsam.Rot3.Logmap(eTo.rotation()),
        eTo.translation(),
        K,
        dist)[0].reshape((-1, 2))
    rrmse += np.sum(np.linalg.norm(reprojected - image_points[i], axis=1) ** 2)
    count += reprojected.shape[0]
rrmse = np.sqrt(rrmse / count)
print(rrmse)

40.081510468848656


In [30]:
## Graph
hTe = gtsam.Pose3(gtsam.Rot3(
    -0.0253614,  0.017791,  0.99952,
    -0.70722,   -0.706973, -0.00536091,   
     0.706539,  -0.707016,  0.030512),
    [0.0839532,  0.144079,  0.0584548])

oTw = gtsam.Pose3(gtsam.Rot3(
     0.0372749, -0.999126,  0.0188992, 
     0.0969558,  0.022439,  0.995036, 
    -0.99459,   -0.0352575, 0.0977075), 
    [0.134352, -0.842135, 1.1393])

rrmse = 0
count = 0
for i in range(len(wTh_list)):
    oTe = oTw.compose(wTh_list[i]).compose(hTe)
    eTo = oTe.inverse()
    reprojected = cv2.projectPoints(
        object_points,
        gtsam.Rot3.Logmap(eTo.rotation()),
        eTo.translation(),
        K,
        dist)[0].reshape((-1, 2))
    rrmse += np.sum(np.linalg.norm(reprojected - image_points[i], axis=1) ** 2)
    count += reprojected.shape[0]
rrmse = np.sqrt(rrmse / count)
print(rrmse)

109.97048559074658


# Swift

In [22]:
with open('out.yaml') as f:
    data = yaml.load(f)

fx = data['camera_calibration']['fx']
fy = data['camera_calibration']['fy']
s = data['camera_calibration']['s']
u0 = data['camera_calibration']['u0']
v0 = data['camera_calibration']['v0']

dist = np.array(data['camera_calibration']['distortion_coefficients'])
K = np.array([[fx, s, u0], [0, fy, v0], [0, 0, 1]])

/home/tuwuhs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


# C++

In [20]:
with open('out.yaml') as f:
    data = yaml.load(f)

fx = data['camera_calibration']['fx']
fy = data['camera_calibration']['fy']
s = data['camera_calibration']['s']
u0 = data['camera_calibration']['u0']
v0 = data['camera_calibration']['v0']

dist = np.array(data['camera_calibration']['distortion_coefficients'])
K = np.array([[fx, s, u0], [0, fy, v0], [0, 0, 1]])

/home/tuwuhs/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  


In [21]:
## Factor graph
hTe = gtsam.Pose3(gtsam.Rot3(
    -0.0222476,  0.00124849,  0.999752,
    -0.70787,   -0.706186,   -0.0148705,
     0.705992,  -0.708025,    0.0165948),
    [0.0977284,  0.148363,    0.0628495])

oTw = gtsam.Pose3(gtsam.Rot3(
     0.0391476, -0.999091,  0.0168659,
     0.0968609,  0.0205936, 0.995085,
    -0.994528,  -0.0373215, 0.097579),
    [0.134747,  -0.848771,  1.14001])

rrmse = 0
count = 0
for i in range(len(wTh_list)):
    oTe = oTw.compose(wTh_list[i]).compose(hTe)
    eTo = oTe.inverse()
    reprojected = cv2.projectPoints(
        object_points,
        gtsam.Rot3.Logmap(eTo.rotation()),
        eTo.translation(),
        K,
        dist)[0].reshape((-1, 2))
    rrmse += np.sum(np.linalg.norm(reprojected - image_points[i], axis=1) ** 2)
    count += reprojected.shape[0]
rrmse = np.sqrt(rrmse / count)
print(rrmse)
print(count)

8.792525108597518
792


# C++ SfM

```
Scale factor: 0.0455747
(gtsam::Pose3)
hTe R: [
	-0.0179847, 0.0032568, 0.999833;
	-0.708825, -0.705306, -0.0104527;
	0.705155, -0.708895, 0.0149932
]
t:  0.112913  0.155531 0.0670113
(gtsam::Pose3)
wTo R: [
	-0.222505, 0.294806, 0.929291;
	-0.974498, -0.095683, -0.202975;
	0.0290792, -0.950755, 0.308578
]
t:  0.785489 0.0910341  0.861706
RRMSE: 12.4806
```

# Sketches, not used

In [9]:
cameras = []
with open('cameras.txt') as f:
    for line in f:
        line = line.strip()
        if line[0] == '#':
            continue
        q = line.split(' ')
        cameras.append([int(q[0]), q[1], float(q[2]), float(q[3]), float(q[4]), float(q[5]), float(q[6]), float(q[7])])
f = cameras[0][4]
u0 = cameras[0][5]
v0 = cameras[0][6]
k1 = cameras[0][7]

K = np.array([[f, 0, u0], [0, f, v0], [0, 0, 1]])
dist = np.array([k1, 0, 0, 0, 0])

In [10]:
points3d = {}
with open('points3D.txt') as f:
    for line in f:
        line = line.strip()
        if line[0] == '#':
            continue
        q = line.split(' ')
        points3d[int(q[0])] = [float(q[1]), float(q[2]), float(q[3])]
object_points = np.array(list(points3d.values()))
object_points_idx = {k: idx for idx, k in enumerate(points3d.keys())}

In [11]:
views = {}
with open('images.txt') as f:
    for line in f:
        line = line.strip()
        if line[0] == '#':
            continue
        q = line.split(' ')
        name = q[9]
        views[name] = []
        line = next(f)
        q = line.split(' ')
        while len(q) > 0:
            u = float(q.pop(0))
            v = float(q.pop(0))
            point3d_id = int(q.pop(0))
            if point3d_id < 0:
                continue
            views[name].append([point3d_id, u, v])

# views[view_idx][point_idx]: point3d_id, u, v
views = [v[1] for v in sorted(views.items(), key = lambda x: x[0])]
# views[0]

In [12]:
## SfM
hTe = gtsam.Pose3(gtsam.Rot3(
-0.0179847, 0.0032568, 0.999833,
-0.708825, -0.705306, -0.0104527,
0.705155, -0.708895, 0.0149932),
[0.112913,  0.155531, 0.0670113])

wTo = gtsam.Pose3(gtsam.Rot3(
-0.222505, 0.294806, 0.929291,
0.974498, -0.095683, -0.202975,
0.0290792, -0.950755, 0.308578),
[0.785489, 0.0910341,  0.861706])

oTw = wTo.inverse()

scale_factor = 0.0455747

rrmse = 0
count = 0
for i in range(len(wTh_list)):
    oTe = oTw.compose(wTh_list[i]).compose(hTe)
    eTo = oTe.inverse()
    reprojected = cv2.projectPoints(
        object_points * scale_factor,
        gtsam.Rot3.Logmap(eTo.rotation()),
        eTo.translation(),
        K,
        dist)[0].reshape((-1, 2))
#     print(reprojected)
    view = views[i]
    for v in view:
        idx = v[0]
        p = np.array(v[1:])
        reproj = reprojected[object_points_idx[idx]]
        print(p, reproj)
        break
    
#     rrmse += np.sum(np.linalg.norm(reprojected - image_points[0], axis=1) ** 2)
#     count += reprojected.shape[0]
# rrmse = np.sqrt(rrmse / count)
# print(rrmse)

[433.888 786.332] [952.81772757 829.73815515]
[ 666.096 1133.43 ] [ 738.68109785 1081.79700723]
[1255.96   511.954] [660.4491777  411.69960955]
[408.115 842.792] [1004.88137979  439.95179044]
[ 472.594 1080.48 ] [643.38970492 671.59673846]
[1300.76   720.808] [579.97181637 206.65542394]
[1032.21 1011.65] [1646.85863565  226.7782703 ]
[497.75  666.309] [ 659.59520257 -294.775382  ]
[1024.84   453.392] [ 118.32092107 -232.53744828]
[577.164 545.187] [1061.19355019  556.2689937 ]
[578.287 571.641] [764.1771967  482.86435587]
[1201.87   546.358] [914.67859026 375.07793387]
[748.229 475.069] [1265.63567708  208.95096392]
[559.973 523.032] [747.82040284 148.2212435 ]
[ 916.331 1111.43 ] [647.48725264 751.4952517 ]
[761.9   491.664] [1376.18040725 -118.16674498]
[1660.78  618.7 ] [1893.68114116 -168.80275586]
[399.899 627.117] [ -4.7186147  202.48703433]
